In [1]:
import torch
import numpy as np
import normflows as nf
import BSCD_simulator

from matplotlib import pyplot as plt
from eZplot import BSCD_plot
from tqdm import tqdm


enable_cuda = True
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BSCD_simulator.adjust_device(device)

model_file = 'BSCD_network.pt'

In [2]:
# Define target
target = BSCD_simulator.target()
context_size = 5

In [3]:
# Define flows
latent_size = 1
hidden_units = 128
hidden_layers = 5

flows = [nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 64,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 128,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 256,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),      
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 256,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size)
         ]

# Set base distribution
q0 = nf.distributions.DiagGaussian(1)#, trainable=False)
    
# Construct flow model
model = nf.ConditionalNormalizingFlow(q0, flows, target)

# Move model on GPU if available
model = model.to(device)

In [4]:
try:
    model.load_state_dict(torch.load(model_file))
    #model.eval()  # Set the model to evaluation mode
    print('loading pretrained network')
except:
    print('starting from scratch')

starting from scratch


In [5]:
max_iter   = 20000
show_iter  = 100
n_batch    = 16

x = target.sample(n=1024*1024)
batch_size = x.size(0)//n_batch

x,context = x[:,0].reshape(-1,1)*1.0,x[:,1:]

batches = torch.arange(x.size(0)).reshape(n_batch,-1)

In [7]:
# Train model
loss_hist = np.array([])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)

In [8]:
for it in tqdm(range(max_iter)):
    loss_epoch = np.array([])

    for it2 in (range(n_batch)):  
        optimizer.zero_grad() 
        # Compute loss
        batch = batches[it2]
        loss = -model.log_prob(x[batch], context[batch]).mean()
        
        # Do backprop and optimizer step
        if ~(torch.isnan(loss) | torch.isinf(loss)):
            loss.backward()
            optimizer.step()
        
        loss_epoch = np.append(loss_epoch, loss.to('cpu').item())

    # Log loss
    loss_hist = np.append(loss_hist, np.mean(loss_epoch))

    if (it+1)%show_iter==0:
        index = (loss_hist.size+np.arange(-int(2.5*show_iter),0))
        BSCD_plot(model,target,loss_hist,index[index>=0])
        #print('yoo')

    if (it + 1) % (3*show_iter) == 0:
        with torch.no_grad():
            torch.save(model.state_dict(), model_file)


    samples_new = target.sample(n=4*1024)
    #replace = torch.randperm(x.size(0))[:4*1024]

    x[:samples_new.size(0)] = samples_new[:,0].reshape((-1,1))
    context[:samples_new.size(0)] = samples_new[:,1:]

    shuffle_index = torch.randperm(x.size(0))
    x = x[shuffle_index]
    context = context[shuffle_index]

 11%|█▏        | 2290/20000 [2:08:18<16:32:18,  3.36s/it]


KeyboardInterrupt: 

In [9]:
torch.save(model.state_dict(), model_file)